# Importation des modules

In [209]:
# Pour manipuler et visualiser les données
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Pour séparer et évaluer les données
from sklearn.model_selection import train_test_split, cross_validate, KFold, learning_curve, GridSearchCV, RandomizedSearchCV

# Pour préparer les données
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import make_column_transformer
from sklearn.pipeline import make_pipeline

# Pour créer des arbres de classification
from sklearn.dummy import DummyClassifier
from sklearn.ensemble import RandomForestClassifier

# Pour faire un modèle de régression logistique 
from sklearn.linear_model import LogisticRegression

# Pour utiliser xgboost
import xgboost as xgb

# Pour utiliser les métriques
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix, classification_report

# Pour exporter notre modèle
import pickle

# Création du dataset et Dummy Classifier

On commence par créer un dataset avec les variables :
<p style='color: #FFA07A'> NAICS, NoEmp, RetainedJob, UrbanRural, Term, UrbanRural, MIS_Status </p>

In [210]:
df = pd.read_csv("archive/SBAnational_clean.csv")

/tmp/ipykernel_6461/3668100114.py:1: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("archive/SBAnational_clean.csv")


In [211]:
# Pour le NAICS, on retient les deux premiers chiffres
df['NAICS'] = df['NAICS'].astype(str).apply(lambda x: x[:2])
df[['NAICS']] = df[['NAICS']].astype(int)

In [212]:
# Dataframe contenant les variables explicatives
X = df[['NAICS', 'NoEmp','RetainedJob', 'UrbanRural','Term']]

# On transforme les valeurs de MIS_Status en 0 ou 1
y = df['MIS_Status'].astype('category').cat.codes

On crée maintenant notre jeu de données d'entraînement et de test :

In [213]:
X_train, X_test, y_train, y_test = train_test_split(X, y, shuffle=True, test_size=0.2, random_state=42, stratify=y)

On sépare les variables numériques et catégorielle :

In [214]:
var_num = ['NoEmp','RetainedJob','Term']

var_cat = ['NAICS', 'UrbanRural']

Puis on crée un transformateur de colonne :

In [215]:
preprocessor = make_column_transformer(
    (StandardScaler(), var_num),
    (OneHotEncoder(), var_cat)
)

On commence notre modélisation par un Dummy Classifier qui servira de point de comparaison :

In [216]:
dummy = make_pipeline(preprocessor, DummyClassifier())

dummy.fit(X_train, y_train)

y_pred = dummy.predict(X_test)

dummy_report = (classification_report(y_test, y_pred))
print(dummy_report)

              precision    recall  f1-score   support

           0       0.00      0.00      0.00     31195
           1       0.82      1.00      0.90    145830

    accuracy                           0.82    177025
   macro avg       0.41      0.50      0.45    177025
weighted avg       0.68      0.82      0.74    177025



/home/apprenant/Documents/Projet_Classification/env/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/apprenant/Documents/Projet_Classification/env/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/apprenant/Documents/Projet_Classification/env/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

# Random Forest Classifier

Nous allons maintenant utiliser un Random Forest Classififier sans paramètres pour le comparer avec notre Dummy Classifier :

In [217]:
# # Entraînement du modèle
# rf_pipe = make_pipeline(preprocessor, RandomForestClassifier())
# rf_pipe.fit(X_train, y_train)

On affiche l'importance des features :

In [218]:
# importances = rf_pipe[-1].feature_importances_
# importances
# # features = pd.DataFrame({"feature": X_train.columns, "importance": importances})
# # features

In [219]:
# rf_pipe.get_feature_names_out

In [220]:
# y_pred = rf_pipe.predict(X_test)

# rfc_report = classification_report(y_test, y_pred)
# print(rfc_report)

En utilisant un Random Forest Classifier sans paramètres, on obtient de bons résultats, proches de notre Dummy Classifier pour les classes positives. <br>
On va essayer d'affiner notre modèle avec en cherchant les meilleurs hyperparamètres :

In [221]:
# # On définit les paramètres à rechercher avec un intervalle de recherche
# param_dist = {'n_estimators': np.arange(20, 100, 1), 
#               'max_depth': np.arange(1, 11),
#               'min_samples_split': np.arange(2, 10),
#               'min_samples_leaf': np.arange(1, 10),
#               'criterion': ['gini','entropy']}

# # On crée un objet Random Search 
# random_search = RandomizedSearchCV(rf_pipe[-1], param_distributions=param_dist,
#                                   cv=5, n_jobs=-1, verbose=2)

# # On entrâine le modèle
# random_search.fit(X_train, y_train)

In [222]:
# # On récupère les meilleurs hyperparamètres trouvés
# best_params = random_search.best_params_
# print(best_params)

In [223]:
# # Entraînement du modèle
# rf_pipe = make_pipeline(preprocessor, RandomForestClassifier(**best_params, class_weight="balanced"))
# rf_pipe.fit(X_train, y_train)

In [224]:
# y_pred = rf_pipe.predict(X_test)

# print(classification_report(y_test, y_pred))

In [225]:
# print(rfc_report)

# Régression logistique

In [226]:
# # Entraînement du modèle
# rf_pipe_log = make_pipeline(preprocessor, LogisticRegression())
# rf_pipe_log.fit(X_train, y_train)

In [227]:
# y_pred = rf_pipe_log.predict(X_test)

# log_report = classification_report(y_test, y_pred)
# print(log_report)

In [ ]:
# # #On définit les paramètres à rechercher avec un intervalle de recherche
# param_dist = {'C': np.logspace(-4, 4, 20),
#               'penalty': ['l1', 'l2', 'elasticnet'],
#               'solver': ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga']}

# # On crée un objet Random Search 
# random_search = RandomizedSearchCV(rf_pipe_log[-1], param_distributions=param_dist,
#                                    n_iter=50, cv=5, n_jobs=-1, verbose=2)

# # On entrâine le modèle
# random_search.fit(X_train, y_train)

In [ ]:
# On récupère les meilleurs hyperparamètres trouvés
best_params = random_search.best_params_
print(best_params)

In [ ]:
# Entraînement du modèle
rf_pipe_log_opti = make_pipeline(preprocessor, LogisticRegression(**best_params, class_weight="balanced"))
rf_pipe_log_opti.fit(X_train, y_train)

In [ ]:
y_pred = rf_pipe_log_opti.predict(X_test)

print(classification_report(y_test, y_pred))
print(log_report)

# XGBoost

In [229]:
# Entraînement du modèle
xgb_pipe = make_pipeline(preprocessor, xgb.XGBClassifier())
xgb_pipe.fit(X_train, y_train)

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('standardscaler',
                                                  StandardScaler(),
                                                  ['NoEmp', 'RetainedJob',
                                                   'Term']),
                                                 ('onehotencoder',
                                                  OneHotEncoder(),
                                                  ['NAICS', 'UrbanRural'])])),
                ('xgbclassifier',
                 XGBClassifier(base_score=None, booster=None, callbacks=None,
                               colsample_bylevel=None, colsample_bynode=None,
                               colsample_bytree=None,
                               early_stoppi...
                               feature_types=None, gamma=None, gpu_id=None,
                               grow_policy=None, importance_type=None,
                               interaction_constraints=None, learning_rate=None,
                               max_bin=None, max_cat_threshold=None,
                               max_cat_to_onehot=None, max_delta_step=None,
                               max_depth=None, max_leaves=None,
                               min_child_weight=None, missing=nan,
                               monotone_constraints=None, n_estimators=100,
                               n_jobs=None, num_parallel_tree=None,
                               predictor=None, random_state=None, ...))])

In [230]:
param_dist = {'n_estimators': np.arange(20, 50, 5), 
              'learning_rate': np.linspace(0.01, 1, 20),
              'max_depth': np.arange(1, 11),
              'subsample': np.linspace(0.1, 1, 10),
              'colsample_bytree': np.linspace(0.1, 1, 10)
             }

random_search = RandomizedSearchCV(xgb_pipe[-1], param_distributions=param_dist,
                                   n_iter=50, cv=5, n_jobs=-1, verbose=2)

random_search.fit(X_train, y_train)

Fitting 5 folds for each of 50 candidates, totalling 250 fits
[CV] END colsample_bytree=0.2, learning_rate=0.5831578947368421, max_depth=10, n_estimators=35, subsample=0.1; total time=  15.1s
[CV] END colsample_bytree=0.2, learning_rate=0.5831578947368421, max_depth=10, n_estimators=35, subsample=0.1; total time=  15.5s
[CV] END colsample_bytree=0.2, learning_rate=0.5831578947368421, max_depth=10, n_estimators=35, subsample=0.1; total time=  15.6s
[CV] END colsample_bytree=0.2, learning_rate=0.5831578947368421, max_depth=10, n_estimators=35, subsample=0.1; total time=  15.6s
[CV] END colsample_bytree=0.2, learning_rate=0.5831578947368421, max_depth=10, n_estimators=35, subsample=0.1; total time=  15.7s
[CV] END colsample_bytree=0.4, learning_rate=0.6873684210526316, max_depth=7, n_estimators=35, subsample=0.7000000000000001; total time=  21.6s
[CV] END colsample_bytree=0.4, learning_rate=0.6873684210526316, max_depth=7, n_estimators=35, subsample=0.7000000000000001; total time=  22.0s


KeyboardInterrupt: 